In [ ]:
print("ok")

In [ ]:
# from langchain_huggingface import HuggingFaceEndpoint
# import re
# import httpx

In [17]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch
# from huggingface_hub import login

In [ ]:
import openai
import re
import httpx

In [ ]:
import os
openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:




class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=self.messages)
        return completion.choices[0].message.content


In [ ]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Django
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris
""".strip()

In [ ]:
action_re = re.compile(r'^Action: (\w+): (.*)')

In [ ]:
def wikipedia(q):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [ ]:
def simon_blog_search(q):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": q,
    })
    return response.json()[0]["text"]

In [ ]:
def calculate(what):
    return eval(what)

In [ ]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [ ]:
def query(question, max_turns=5):
    i = 0
    bot = ChatBot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [ ]:
print(query("What does Pakistan share borders with?"))

In [16]:
# from langchain.llms import HuggingFaceEndpoint
# from huggingface_hub import whoami

# class ChatBot:
#     def __init__(self, 
#                  repo_id="mistralai/Mistral-7B-Instruct-v0.3", 
#                  system="", 
#                  token="", 
#                  temperature=0.7, 
#                  max_length=128):
#         if not token:
#             raise ValueError("A Hugging Face authentication token is required.")

#         # Validate Token
#         try:
#             whoami(token=token)
#         except Exception as e:
#             raise ValueError(f"Invalid Hugging Face token: {e}")

#         self.llm = HuggingFaceEndpoint(
#             repo_id=repo_id,
#             token=token,
#             temperature=temperature,
#             model_kwargs={"max_length": max_length}
#         )

#         self.messages = []
#         if system:
#             self.messages.append({"role": "system", "content": system})

#     def __call__(self, message):
#         self.messages.append({"role": "user", "content": message})
#         try:
#             result = self.execute()
#             self.messages.append({"role": "assistant", "content": result})
#             return result
#         except Exception as e:
#             return f"Error during generation: {e}"

#     def execute(self):
#         conversation = "".join(
#             f"{msg['role'].capitalize()}: {msg['content']}\n" for msg in self.messages
#         )
#         conversation += "Assistant:"
#         try:
#             response = self.llm(conversation)
#             return response.strip()
#         except Exception as e:
#             raise RuntimeError(f"Error during generation: {e}")
